In [4]:
import pandas as pd
import numpy as np
import talib
import csv
from pandas_datareader import data
from datascience import *
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn import model_selection
from sklearn.model_selection import train_test_split
%matplotlib inline

In [5]:
amzn = pd.read_csv("AMZN12822.csv")

In [6]:
amzn['EMA30'] = talib.EMA(amzn['Close'].values, timeperiod=30)
amzn['UPPERBAND'], amzn['MIDDLEBAND'], amzn['LOWERBAND'] = talib.BBANDS(amzn['Close'].values, timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)

In [4]:
amzn['NextDayPrice'] = amzn['Close'].shift(-1)

In [5]:
amzn = amzn.replace("-","", regex=True)
amzn_clean = amzn.copy()
amzn_clean.dropna(inplace=True)
print (amzn_clean.head())

        Date         Open         High          Low        Close    Adj Close  \
49  20210412  3355.209961  3395.040039  3351.149902  3379.389893  3379.389893   
50  20210413  3400.850098  3432.000000  3395.629883  3400.000000  3400.000000   
51  20210414  3404.040039  3404.129883  3326.000000  3333.000000  3333.000000   
52  20210415  3371.000000  3397.000000  3352.000000  3379.090088  3379.090088   
53  20210416  3380.000000  3406.800049  3355.590088  3399.439941  3399.439941   

       Volume        EMA50    UPPERBAND   MIDDLEBAND    LOWERBAND  \
49  3281800.0  3174.635986  3362.711955  3150.644006  2938.576057   
50  3315800.0  3183.473791  3402.025531  3166.560010  2931.094488   
51  3145200.0  3189.337564  3422.106155  3178.617004  2935.127854   
52  3233600.0  3196.778839  3448.398591  3190.785010  2933.171429   
53  3186000.0  3204.726333  3470.876032  3209.357507  2947.838982   

    NextDayPrice  
49   3400.000000  
50   3333.000000  
51   3379.090088  
52   3399.439941  
53 

In [6]:
X_all = amzn_clean.iloc[:, amzn_clean.columns != 'NextDayPrice']  # feature values for all days
y_all = amzn_clean['NextDayPrice']  # corresponding targets/labels
X_all = X_all.replace("-","", regex=True)
print (X_all.head()) 

#Splitting the data into training and testing sets using the given feature as the target
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.30, random_state=42)

        Date         Open         High          Low        Close    Adj Close  \
49  20210412  3355.209961  3395.040039  3351.149902  3379.389893  3379.389893   
50  20210413  3400.850098  3432.000000  3395.629883  3400.000000  3400.000000   
51  20210414  3404.040039  3404.129883  3326.000000  3333.000000  3333.000000   
52  20210415  3371.000000  3397.000000  3352.000000  3379.090088  3379.090088   
53  20210416  3380.000000  3406.800049  3355.590088  3399.439941  3399.439941   

       Volume        EMA50    UPPERBAND   MIDDLEBAND    LOWERBAND  
49  3281800.0  3174.635986  3362.711955  3150.644006  2938.576057  
50  3315800.0  3183.473791  3402.025531  3166.560010  2931.094488  
51  3145200.0  3189.337564  3422.106155  3178.617004  2935.127854  
52  3233600.0  3196.778839  3448.398591  3190.785010  2933.171429  
53  3186000.0  3204.726333  3470.876032  3209.357507  2947.838982  


In [7]:
 from sklearn.linear_model import LinearRegression
 
 
#Create a decision tree regressor and fit it to the training set
 regressor = LinearRegression()
 
 regressor.fit(X_train,y_train)
 
 print ("Training set: {} samples".format(X_train.shape[0]))
 print ("Test set: {} samples".format(X_test.shape[0]))

Training set: 140 samples
Test set: 61 samples


In [8]:
scores = model_selection.cross_val_score(regressor, X_test, y_test, cv=10)
print ("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() / 2))    

from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, regressor.predict(X_test))
print("MSE: %.4f" % mse)

Accuracy: 0.59 (+/- 0.16)
MSE: 3809.7806


In [9]:
last_row = X_all[-1:]
print(last_row)

         Date         Open    High          Low        Close    Adj Close  \
249  20220125  2844.850098  2872.0  2762.899902  2799.719971  2799.719971   

        Volume        EMA50    UPPERBAND   MIDDLEBAND  LOWERBAND  
249  4541200.0  3294.735352  3570.556008  3212.779004   2855.002  


In [10]:
print(regressor.predict(last_row))

[2724.6437718]
